# 04 Run Fleet (Alternative Solution)

## Introduction

SageMaker Edge Manager is a service from Amazon SageMaker that lets you:

+ prepare custom models for edge device hardware
+ include a runtime for running machine learning inference efficiently on edge devices
+ enable the device to send samples of data from each model securely to SageMaker for relabeling and retraining.

There are two main components to this service:
+ SageMaker Edge Manager in the Cloud 
+ SageMaker Edge Agent on the Edge device

This notebook demonstrates the end-to-end workflow for getting a running SageMaker Edge on the edge device. This will involve the following steps:

+ Compile the model using SageMaker Neo
+ Package the compiled model with SageMaker Edge Manager
+ Deploy with SageMaker Edge Manager Agent
+ Run inference with the model
+ Capture model's input and output data to S3

**Note**:
Typically, the SageMaker Edge Agent is run on an Edge device. For the sake of this notebook, we will run the Agent on an EC2 instance. We show how to package the compiled model and then load it to the Agent on the Edge Device to make predictions with. Finally, we show how to capture model's input and output to S3 via the Agent.

 When you run this notebook, choose the kernel: `conda_tensorflow_p36` if you are using a notebook instance or `Python 3 (Data Science)` if you are using SageMaker Studio.

**Please note**: There are pricing implications to the use of this notebook. Please refer to [Edge Manager](https://aws.amazon.com/sagemaker/edge-manager/pricing) for more information.

## Demo Setup

We need an AWS account role with SageMaker access. This role is used to give SageMaker access to S3, launch an EC2 instance and create components and deployments in Greengrass.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json
import time

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [ ]:
print(role)

Locate the above printed sagemaker role from [IAM console](https://console.aws.amazon.com/iam), find and attach the following policies to role:

- IAMFullAccess

You can find more information about how to attach policies to role [here](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-attach-detach.html#add-policies-console).

Also add the following trust relationship
```
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "iam.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
```

In [ ]:
iam_client = boto3.client('iam')
role_name = role.split('/')[-1]

In [ ]:
# use python sdk to attach a few more managed policy to sagemaker role
policy_attach_res = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/AmazonEC2FullAccess"
)

policy_attach_res = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/service-role/AmazonEC2RoleforSSM"
)

policy_attach_res = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore"
)

policy_attach_res = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/AmazonSSMFullAccess"
)

policy_attach_res = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/AWSGreengrassFullAccess"
)

policy_attach_res = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/AWSIoTFullAccess"
)

We then need an S3 bucket that would be used for storing the model artifacts generated after compilation and packaged artifacts generated after edge packaging job.

In [ ]:
# S3 bucket and folders for saving model artifacts.
# Feel free to specify different bucket/folders here if you wish.
bucket = sess.default_bucket()
folder = "DEMO-Sagemaker-Edge"
compilation_output_sub_folder = folder + "/compilation-output"
iot_folder = folder + "/iot"

# S3 Location to save the model artifact after compilation
s3_compilation_output_location = "s3://{}/{}".format(bucket, compilation_output_sub_folder)

Finally, we upload the test image to S3 bucket. This image will be used in inference later.

### Launch EC2 Instance

As mentioned earlier, this EC2 instance is used in place of an Edge device for running the agent software.

In [ ]:
ec2_client = boto3.client("ec2", region_name=region)

Generate key pair for EC2 instance, save the key PEM file. We can use this key with SSH to connect to the instance. But in this notebook example, we will not use SSH, instead, we will use AWS Systems Manager to send commands to the instance.

In [ ]:
key_pairs = ec2_client.describe_key_pairs()
key_names = list(map(lambda x: x["KeyName"], key_pairs["KeyPairs"]))

key_name = "ec2-key-pair"

if key_name in key_names:
    ec2_key_pair = ec2_client.delete_key_pair(
        KeyName=key_name,
    )

In [ ]:
ec2_key_pair = ec2_client.create_key_pair(
    KeyName=key_name,
)

key_pair = str(ec2_key_pair["KeyMaterial"])
key_pair_file = open("ec2-key-pair.pem", "w")
key_pair_file.write(key_pair)
key_pair_file.close()

In [ ]:
ec2_role_name = "SMEdgeManageExampleRole"

trust_relationship_ec2_service = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "ec2.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [ ]:
# create EC2 role and its instance profile
from botocore.exceptions import ClientError

try:
    create_role_res = iam_client.create_role(
        RoleName=ec2_role_name,
        AssumeRolePolicyDocument=json.dumps(trust_relationship_ec2_service),
        Description='This is a EC2 role',
    )
except ClientError as error:
    if error.response['Error']['Code'] == 'EntityAlreadyExists':
        print('Role already exists... hence exiting from here')
    else:
        print('Unexpected error occurred... Role could not be created', error)

In [ ]:
policy_attach_res = iam_client.attach_role_policy(
    RoleName=ec2_role_name,
    PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess"
)

policy_attach_res = iam_client.attach_role_policy(
    RoleName=ec2_role_name,
    PolicyArn="arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore"
)

policy_attach_res = iam_client.attach_role_policy(
    RoleName=ec2_role_name,
    PolicyArn="arn:aws:iam::aws:policy/CloudWatchAgentAdminPolicy"
)

In [ ]:
# specify your role name used by GGv2
iot_device_role_name = 'SageMakerTESRole'

Create a role for the EC2 instance we are going to use. Read for detailed information about [IAM roles for Amazon EC2](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/iam-roles-for-amazon-ec2.html).

In [ ]:
account_id = role.split(":")[4]

# Create a policy
my_managed_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "CreateTokenExchangeRole",
            "Effect": "Allow",
            "Action": [
                "iam:AttachRolePolicy",
                "iam:CreatePolicy",
                "iam:CreateRole",
                "iam:GetPolicy",
                "iam:GetRole",
                "iam:PassRole"
            ],
            "Resource": [
                f"arn:aws:iam::{account_id}:role/{iot_device_role_name}",
                f"arn:aws:iam::{account_id}:policy/{iot_device_role_name}Access",
                f"arn:aws:iam::aws:policy/{iot_device_role_name}Access"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "iot:AddThingToThingGroup",
                "iot:AttachPolicy",
                "iot:AttachThingPrincipal",
                "iot:CreateKeysAndCertificate",
                "iot:CreatePolicy",
                "iot:CreateRoleAlias",
                "iot:CreateThing",
                "iot:CreateThingGroup",
                "iot:DescribeEndpoint",
                "iot:DescribeRoleAlias",
                "iot:DescribeThingGroup",
                "iot:GetPolicy",
                "sts:GetCallerIdentity"
            ],
            "Resource": "*"
        },
        {
            "Sid": "DeployDevTools",
            "Effect": "Allow",
            "Action": [
                "greengrass:CreateDeployment",
                "iot:CancelJob",
                "iot:CreateJob",
                "iot:DeleteThingShadow",
                "iot:DescribeJob",
                "iot:DescribeThing",
                "iot:DescribeThingGroup",
                "iot:GetThingShadow",
                "iot:UpdateJob",
                "iot:UpdateThingShadow"
            ],
            "Resource": "*"
        }
    ]
}

response = iam_client.create_policy(
  PolicyName='my_ggv2_provision_policy',
  PolicyDocument=json.dumps(my_managed_policy)
)

In [ ]:
policy_attach_res = iam_client.attach_role_policy(
    RoleName=ec2_role_name,
    PolicyArn=response['Policy']['Arn']
)

In [ ]:
pass_ec2_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": f"arn:aws:iam::{account_id}:role/{ec2_role_name}"
        }
    ]
}

response = iam_client.create_policy(
  PolicyName='my_pass_ec2_role_policy',
  PolicyDocument=json.dumps(pass_ec2_role_policy)
)

In [ ]:
import boto3

client = boto3.client('iam')

response = client.create_instance_profile(
    InstanceProfileName=ec2_role_name
)

Launch an EC2 C5 instance. In this example we will use aws deep learning AMI.

In [ ]:
ami = ec2_client.describe_images(Filters=[{'Name': 'name', 'Values': ['Deep Learning AMI (Ubuntu 18.04) Version 36.0']}])['Images'][0]['ImageId']
ami

In [ ]:
iam_resource = boto3.resource('iam')
instance_profile = iam_resource.InstanceProfile(
    ec2_role_name
)

instance_profile.add_role(
    RoleName=ec2_role_name
)

print('IAM role has been added to the Instance Profile')

In [ ]:
ec2_profile_name = ec2_role_name  # the name of the role created for EC2

ec2_instance = ec2_client.run_instances(
    ImageId=ami,
    MinCount=1,
    MaxCount=1,
    InstanceType="c5.large",
    KeyName=key_name,
    IamInstanceProfile={"Name": ec2_profile_name},
)

In [ ]:
instance_id = ec2_instance["Instances"][0]["InstanceId"]  # will used for running inference later
print(instance_id)

## Compile Model using SageMaker Neo


In [ ]:
sm_client = boto3.client("sagemaker", region_name=region)

In [ ]:
model_file = open("model/model.tar.gz", "rb")
boto3.Session().resource("s3").Bucket(bucket).Object('model/model.tar.gz').upload_fileobj(model_file)
print("Model successfully uploaded!")

In [ ]:
compilation_job_name = 'wind-turbine-anomaly-%d' % int(time.time()*1000)
sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': 's3://%s/model/model.tar.gz' % bucket,
        'DataInputConfig': '{"input0":[1,6,10,10]}',
        'Framework': 'PYTORCH'
    },
    OutputConfig={
        'S3OutputLocation': 's3://%s/wind_turbine/optimized/' % bucket,        
        'TargetPlatform': { 'Os': 'LINUX', 'Arch': 'X86_64' }
    },
    StoppingCondition={ 'MaxRuntimeInSeconds': 900 }
)
while True:
    resp = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)    
    if resp['CompilationJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['CompilationJobStatus'], compilation_job_name)
        break
    time.sleep(5)

### Package Keras Model

In [ ]:
import time
model_version = '1.0'
model_name = 'WindTurbineAnomalyDetection'
model_component_version = "1.0.0"
model_component_name = "com.model.WindTurbineAnomalyDetection"

edge_packaging_job_name='wind-turbine-anomaly-%d' % int(time.time()*1000)
resp = sm_client.create_edge_packaging_job(
    EdgePackagingJobName=edge_packaging_job_name,
    CompilationJobName=compilation_job_name,
    ModelName=model_name,
    ModelVersion=model_version,
    RoleArn=role,
    OutputConfig={
        "PresetDeploymentType": "GreengrassV2Component",
        "PresetDeploymentConfig": json.dumps(
            {"ComponentName": model_component_name, "ComponentVersion": model_component_version}
        ),
        'S3OutputLocation': s3_compilation_output_location
    }
)

# Poll every 30 sec
while True:
    job_status = sm_client.describe_edge_packaging_job(
        EdgePackagingJobName=edge_packaging_job_name
    )
    if job_status["EdgePackagingJobStatus"] == "COMPLETED":
        break
    elif job_status["EdgePackagingJobStatus"] == "FAILED":
        raise RuntimeError("Edge Packaging failed")
    print("Packaging ...")
    time.sleep(30)


### Install Greengrass

SageMaker Edge Manager can use AWS IoT Greengrass to deploy the agent, the model and the inference application to the edge device.

AWS IoT Greengrass provides all the necessary features to manage applications on remote devices in a secure and scalable way. To learn more about Greengrass, head to the [What is AWS IoT Greengrass?](https://docs.aws.amazon.com/greengrass/v2/developerguide/what-is-iot-greengrass.html). 

The SageMaker Edge Manager agent leverages the AWS credentials provided by the [Token exchange service](https://docs.aws.amazon.com/greengrass/v2/developerguide/token-exchange-service-component.html) component to securely communicate with the SageMaker Edge Manager backend.


In [ ]:
ssm_client = boto3.client("ssm", region_name=region)

**Note**: If you are using a real device, connect to the device via SSH, ensure that you have both Java v8 or above and the Unzip command and then run the following commands (replace `<your_region>` with the correct value). To run this command on the device you also need to provide IAM credentials with at least the permissions listed in [Minimal IAM policy for installer to provision resources](https://docs.aws.amazon.com/greengrass/v2/developerguide/provision-minimal-iam-policy.html).

```bash
curl -s https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip > greengrass-nucleus-latest.zip \
               && unzip greengrass-nucleus-latest.zip -d GreengrassCore,
sudo -E java -Droot="/greengrass/v2" -Dlog.store=FILE -jar ./GreengrassCore/lib/Greengrass.jar \
    --thing-name GreengrassSMEdgeManagerDevice -trn SageMakerTESRole -tra SageMakerTESRoleAlias \ 
    --thing-group-name GreengrassSMEdgeManagerGroup \ 
    --component-default-user ggc_user:ggc_group --provision true --setup-system-service true --deploy-dev-tools true \
    --aws-region <your_region>
```

Otherwise, run the following command to install Greengrass on the EC2 instance. You can find further information about the command to run and their effect in [Install AWS IoT Greengrass Core software with automatic resource provisioning](https://docs.aws.amazon.com/greengrass/v2/developerguide/quick-installation.html).

In [ ]:
device_name = "demo-device" + str(time.time()).split(".")[0]

In [ ]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "sudo apt update && apt install python3-venv -y",
            "curl -s https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip > greengrass-nucleus-latest.zip && unzip greengrass-nucleus-latest.zip -d GreengrassCore",
            f'sudo -E java -Droot="/greengrass/v2" -Dlog.store=FILE -jar ./GreengrassCore/lib/Greengrass.jar --aws-region {region} --thing-name {device_name} -trn {iot_device_role_name} -tra SageMakerTESRoleAlias --thing-group-name GreengrassSMEdgeManagerGroup --component-default-user ggc_user:ggc_group --provision true --setup-system-service true --deploy-dev-tools true',
        ]
    },
)

In [ ]:
ssm_client.get_command_invocation(
    CommandId=response["Command"]["CommandId"],
    InstanceId=instance_id,
)

### Create Device Fleet

#### Modify the IAM role for device fleet

In [ ]:
# Create a policy
download_component_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::*SageMaker*",
                "arn:aws:s3:::*Sagemaker*",
                "arn:aws:s3:::*sagemaker*"
            ]
        }
    ]
}

response = iam_client.create_policy(
  PolicyName='download_component_policy',
  PolicyDocument=json.dumps(download_component_policy)
)

In [ ]:
role_arn = f"arn:aws:iam::{account_id}:role/{iot_device_role_name}"

In [ ]:
policy_attach_res = iam_client.attach_role_policy(
    RoleName=iot_device_role_name,
    PolicyArn=response['Policy']['Arn']
)

policy_attach_res = iam_client.attach_role_policy(
    RoleName=iot_device_role_name,
    PolicyArn="arn:aws:iam::aws:policy/service-role/AmazonSageMakerEdgeDeviceFleetPolicy"
)

In [ ]:
response = iam_client.update_assume_role_policy(
    PolicyDocument='''{
        "Version": "2012-10-17",
        "Statement": [
           {
             "Effect": "Allow",
             "Principal": {"Service": "credentials.iot.amazonaws.com"},
             "Action": "sts:AssumeRole"
           },
           {
             "Effect": "Allow",
             "Principal": {"Service": "sagemaker.amazonaws.com"},
             "Action": "sts:AssumeRole"
           }
        ]
    }''',
    RoleName=iot_device_role_name,
)

In [ ]:
device_fleet_name = "demo-device-fleet" + str(time.time()).split(".")[0]

sm_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn=role_arn,
    OutputConfig={"S3OutputLocation": s3_compilation_output_location},
)

print(device_fleet_name)

#### Register device to the fleet

In [ ]:
sm_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {
            "DeviceName": device_name,
            "IotThingName": device_name,
            "Description": "this is a sample virtual device",
        }
    ],
)

## Inference on Edge

In this example, we will use [AWS IoT Greengrass](https://docs.aws.amazon.com/) to remotely deploy the agent, the model and the inference application.

The [SageMaker Edge Manager component](https://docs.aws.amazon.com/greengrass/v2/developerguide/sagemaker-edge-manager-component.html) is already provided and will be used to deploy and run the agent on the device.

The model component has been created for you by the packaging jobs you execute previously and in [your account](https://console.aws.amazon.com/iot/home?#/greengrass/v2/components) you should now have 1 component called `com.model.keras`. 

In order to be able to use the model, we also need an application component to load the model and invoke it. In the next section we are going to see how to create such component.

## Create the inference application component

We will use a Python application to load the model and perform the inference. The application is provided in the [inference.py](./inference.py) file. In addition to this file, you will also need to generate the Protobuf libraries that can be used with the gRPC API of the agent.

First list the available releases from the S3 bucket. It does not matter which OS we are going to use since we only need the protobuf definitions.

### Create the AWS IoT Greengrass component

Next, upload the python script, dataset and the protobuf libraries to an S3 bucket so that they can be referenced by the component recipe and from where they will be downloaded by Greengrass on the device. We use the SageMaker bucket for this.

In [ ]:
!mkdir -p data
!curl https://aws-ml-blog.s3.amazonaws.com/artifacts/monitor-manage-anomaly-detection-model-wind-turbine-fleet-sagemaker-neo/dataset_wind_turbine.csv.gz -o data/dataset_wind.csv.gz

In [ ]:
folder = "com.sagemaker.windturbine/1.0.0"
sess.upload_data("app_ggv2/entry_point.py", bucket, folder)
sess.upload_data("app_ggv2/agent_pb2.py", bucket, folder)
sess.upload_data("app_ggv2/agent_pb2_grpc.py", bucket, folder)
sess.upload_data("app_ggv2/edgeagentclient.py", bucket, folder)
sess.upload_data("app_ggv2/turbine.py", bucket, folder)
sess.upload_data("app_ggv2/windfarm.py", bucket, folder)
sess.upload_data("data/dataset_wind.csv.gz", bucket, folder)
sess.upload_data("statistics/raw_std.npy", bucket, folder)
sess.upload_data("statistics/mean.npy", bucket, folder)
sess.upload_data("statistics/std.npy", bucket, folder)
sess.upload_data("statistics/thresholds.npy", bucket, folder)

As a final step, create the component. You can use the [AWS Greengrass Console](https://console.aws.amazon.com/iot/home?#/greengrass/v2/components/create) to create a new component. Go to **Component>Create component**, select `Enter recipe as YAML` and copy and paste the following YAML. 

In [ ]:
recipe = f'''
---
RecipeFormatVersion: 2020-01-25
ComponentName: com.sagemaker.windturbine
ComponentVersion: 1.0.0
ComponentDescription: Sagemaker Edge Manager Python example
ComponentPublisher: Amazon Web Services, Inc.
ComponentDependencies:
  aws.greengrass.SageMakerEdgeManager:
    VersionRequirement: '>=1.0.0'
    DependencyType: HARD
  com.model.WindTurbineAnomalyDetection:
    VersionRequirement: '~1.0.0'
    DependencyType: HARD
Manifests:
  - Platform:
      os: linux
      architecture: "/amd64|x86/"
    Lifecycle:
      Install: |-
        python3 -m venv venv
        . venv/bin/activate
        pip install pip --upgrade
        pip install wheel      
        pip3 install grpcio==1.38.1
        pip3 install grpcio-tools==1.38.1
        pip3 install protobuf
        pip3 install Pillow
        pip3 install numpy
        pip3 install pandas
        pip3 install PyWavelets
      Run:
        Script: |- 
          export MODEL_PATH="{{com.model.WindTurbineAnomalyDetection:work:path}}"
          export ARTIFACTS_PATH="{{artifacts:path}}"
          . venv/bin/activate  
          python3 -u {{artifacts:path}}/entry_point.py 
    Artifacts:
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/entry_point.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/agent_pb2.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/agent_pb2_grpc.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/edgeagentclient.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/turbine.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/windfarm.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/dataset_wind.csv.gz
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/raw_std.npy
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/mean.npy
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/std.npy
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/thresholds.npy
'''

print(recipe)

# Deploy the application

Once the application component has been created, it can be deployed to the device. 

1. In the [AWS Console](https://console.aws.amazon.com/iot/home#/greengrass/v2/components/private) select the `com.sagemaker.edgePythonExample` component, and choose **Deploy**.
2. Select the deployment named `Deployment for GreengrassSMEdgeManagerGroup` and choose `Next`
3. Toggle the selector on the `Public components` pane, search for `SageMakerEdgeManager`, and selct it
4. Choose **Next**
5. Select the `aws.greengrass.SageMakerEdgeManager` component and choose **Configure component**
6. Replace the **Configuration to merge** content with the following json. Don't forget to change the placeholder to the actual values.
```json
{
	"DeviceFleetName": <device-fleet-name>,
	"BucketName": <bucket-name>
}
```
7. Choose **Confirm**
3. Choose **Next** until you reach the last screen. 
4. Choose **Deploy**.

You can check the status of the deployment by clicking on the device name `GreengrassSMEdgeManagerDevice` in the Target core devices pane and then selecting Deployments tab.

After few seconds the components will have been deployed to the EC2 instance and you should be able to see the results of the inference in the Greengrass logs by executing:
```bash
sudo cat /greengrass/v2/logs/com.sagemaker.edgePythonExample.log
```

If you are using another device, it might take longer depending on the network speed.

As you noticed there is no need to install the agent separately or download the model: these tasks are performed by Greengrass based on the fact that the application recipe defined the dependencies on SageMagerEdge manager component and the model component created by the packaging job.



In [ ]:
cat_log_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "sudo tail -30 /greengrass/v2/logs/com.sagemaker.windturbine.log",
        ]
    },
)

In [ ]:
output = ssm_client.get_command_invocation(
    CommandId=cat_log_out["Command"]["CommandId"],
    InstanceId=instance_id,
)["StandardOutputContent"]
print(output)

## Clean Up

Undeploy the application

In [ ]:
ssm_client.cancel_command(CommandId=agent_out["Command"]["CommandId"], InstanceIds=[instance_id])

Stop the EC2 instance

In [ ]:
ec2_client.stop_instances(InstanceIds=[instance_id])

Detach and delete policy

In [ ]:
iot_client.detach_policy(policyName=policy_name, target=iot_cert["certificateArn"])

iot_client.delete_policy(policyName=policy_name)

Deregister device and delete device fleet

In [ ]:
sagemaker_client.deregister_devices(DeviceFleetName=device_fleet_name, DeviceNames=[device_name])

sagemaker_client.delete_device_fleet(DeviceFleetName=device_fleet_name)